In [1]:
import sys, random, string, re, time, os


In [2]:
%%capture
!pip install git+https://github.com/huggingface/transformers.git  -U 
!pip install git+https://github.com/huggingface/accelerate.git  -U 
!pip install bitsandbytes 
!pip install git+https://github.com/huggingface/peft.git  -U 

In [3]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch
from peft import PeftModel, PeftConfig


bnb_config = BitsAndBytesConfig(  
    load_in_4bit= True,
    bnb_4bit_quant_type= "nf4",
    bnb_4bit_compute_dtype= torch.float16,
    bnb_4bit_use_double_quant= True,
llm_int8_enable_fp32_cpu_offload= True)

In [4]:
torch.cuda.empty_cache()

In [5]:
import gc
gc.collect()

30

## Get the Model

In [6]:
model = AutoModelForCausalLM.from_pretrained(
        '/kaggle/input/mixtral/pytorch/8x7b-instruct-v0.1-hf/1',
        quantization_config=bnb_config,
        device_map="auto",
        trust_remote_code=True,)

Loading checkpoint shards:   0%|          | 0/19 [00:00<?, ?it/s]

## Get the tokenizer

In [7]:
from transformers import pipeline, AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained('/kaggle/input/mixtral/pytorch/8x7b-instruct-v0.1-hf/1', trust_remote_code=True)

2024-04-15 13:04:11.677171: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-15 13:04:11.677294: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-15 13:04:11.863023: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [8]:
import gc
gc.collect()

30

## Model Inference

In [9]:
abstract = 'Diabetes is now one of the major public health challenges, globally. Prolonged diabetes leads to various diabetic microvascular complications (DMCs) like retinopathy, nephropathy, and neuropathy. Multiple factors are likely to be involved in predisposing diabetic individuals to complications. Early detection or diagnosis is essential in developing strategies to reduce the risk factors and management costs of these diabetic complications. In this study, we employed Raman Spectroscopy (RS) to analyse the plasma samples of diabetes patients without and with DMCs along with the plasma samples of healthy subjects. Spectral comparisons revealed decrease in protein content in Diabetes group and further subsequent decrease in proteins in DMC groups when compared with control group, which corroborates with the fact that there exists increased secretion of proteins in urine and corresponding decreased protein content in their blood in case of diabetic individuals. Among all study groups, it was noted that 75% of control spectra show correct classification, while spectral misclassification is high amongst the subjects with Diabetes and DMCs. Interestingly, very few Diabetes and DMC plasma spectra are misclassified as control spectra. Findings demonstrate that 70% of the Diabetes subjects without complications can be correctly identified from diabetes with complications. Further, investigations could also attempt to explore the use of serum instead of plasma to reduce the spectral misclassifications as one of the abundant constituents namely clotting factors could be avoided. The outcome of RS study may be imminent for the early detection or diagnosis of DMCs.'

example = {'instruction' : 'Write 5 catchy title each not more than 15 words for the following paper abstract. The title should be a single sentence that accurately captures what you have done and sounds interesting to the people who work on the same or a similar topic. The title should contain the important title keywords that other researchers use when looking for literature in databases. The title should also use synonyms, broader terms, or abstractive keywords to make it more appealing and informative. Do not use words that are not related to the paper extract or the topic',
    'input' : abstract }
def formatting_func(example):
    text = f"<s>[INST]Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.{example['instruction']}\n\n### Input:{example['input']}[/INST]"
    return text
eval_prompt = formatting_func(example)
print(eval_prompt)

<s>[INST]Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.Write 5 catchy title each not more than 15 words for the following paper abstract. The title should be a single sentence that accurately captures what you have done and sounds interesting to the people who work on the same or a similar topic. The title should contain the important title keywords that other researchers use when looking for literature in databases. The title should also use synonyms, broader terms, or abstractive keywords to make it more appealing and informative. Do not use words that are not related to the paper extract or the topic

### Input:Diabetes is now one of the major public health challenges, globally. Prolonged diabetes leads to various diabetic microvascular complications (DMCs) like retinopathy, nephropathy, and neuropathy. Multiple factors are likely to be involved in predisposing diabetic ind

<div class="anchor" id="top" style="
    margin-right: auto; 
    margin-left: auto;
    padding: 10px;
   font-size : 120%;
    background-color: #FEF2EF;
    border-radius: 2px;
    font-color :  #581845  ;        
    border: 2px solid #FF5733 ;"> 

#### To filter out any unnecessary warnings

In [10]:
import warnings
warnings.filterwarnings("ignore")
warnings.filterwarnings("ignore", category=UserWarning, module="transformers")

In [11]:
from datetime import datetime
import re

def model_seq_gen(model) : 
        pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer)
        start = datetime.now()
        sequences = pipe(
            f'{eval_prompt}' ,
            do_sample=True,
            max_new_tokens=200, 
            temperature=0.7, 
            top_p=0.95
        )
        extracted_title = re.sub(r'[\'"]', '', sequences[0]['generated_text'].split("[/INST]")[1])
        stop = datetime.now()
        time_taken = stop-start
        print(f"Execution Time : {time_taken}")
        return extracted_title
'''
extracted_title = model_seq_gen(model)
print(extracted_title)
'''

'\nextracted_title = model_seq_gen(model)\nprint(extracted_title)\n'

In [12]:
!pip install faker


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 6.9 MB/s eta 0:00:00


In [13]:
!pip install fpdf


  Preparing metadata (setup.py) ... - \ done
  Created wheel for fpdf: filename=fpdf-1.7.2-py2.py3-none-any.whl size=40703 sha256=cb35eb27cefb85f0d47461328ff5d25e18c3d0f3f1e3e47910024d4664f4516e
  Stored in directory: /root/.cache/pip/wheels/f9/95/ba/f418094659025eb9611f17cbcaf2334236bf39a0c3453ea455
Successfully built fpdf


In [14]:
from fpdf import FPDF
import numpy as np
import random

In [15]:
company_names = np.array(['1. Allstate Insurance Company',

'State Farm Insurance',
'Progressive Corporation',
'Liberty Mutual Insurance',
'Geico',
'Farmers Insurance Group',
'Travelers Insurance',
'Nationwide Mutual Insurance Company','American Family Insurance',
'USAA',
'Chubb Limited',
'Hartford Financial Services Group',
'AIG (American International Group)',
'MetLife',
'The Hartford',
'The Travelers Companies',
'Zurich Insurance Group',
'Cigna',
'UnitedHealthcare',
'Aetna',
'Blue Cross Blue Shield Association',
'Anthem, Inc.',
'Humana',
'UnitedHealth Group',
'Kaiser Permanente',
'Health Care Service Corporation',
'CVS Health',
'United States Fire Insurance Company',
'The Hanover Insurance Group',
'The Hartford Financial Services Group, Inc.',
'The Cincinnati Insurance Company'])

In [16]:
from random import randrange
premium_amount = randrange(10000,1000000)
print(premium_amount)

241023


In [17]:
from faker import Faker

In [18]:
    fake = Faker()
    primary_name = fake.name()
    primary_address = fake.address()
    primary_city = fake.city()
    primary_state = fake.state()
    primary_zip = fake.zipcode()
    primary_email_address   = fake.email()
    secondary_name = fake.name()
    secondary_address = fake.address()
    secondary_email_address   = fake.email()
    company_name = random.choice(company_names)
    contract_id = random.choice([fake.password(length = 16, special_chars = False ),fake.uuid4()])
    a1 = fake.date_of_birth()
    b1  = fake.date_of_birth()
    
    if a1 < b1:
        begin_date = str(a1.year)+'-'+str(a1.month)+'-'+str(a1.day)
        end_date   =   str(b1.year)+'-'+str(b1.month)+'-'+str(b1.day)
    else:
        begin_date = str(b1.year)+'-'+str(b1.month)+'-'+str(b1.day)
        end_date   =   str(a1.year)+'-'+str(a1.month)+'-'+str(a1.day)


    
    #prompt enginer 
    prompt = "Compose a standard Legal Auto Claims Insurance Policy document between the Insurer {} and {} \. The following are the details of the Insurer. \
    \nAddress: {} \nEmail:   {}. The Policy Number  for the Legal Auto Claims Policy is {}. \
    \nThe Start date of the Legal Auto Claims Policy is {} and the End date of Legal Auto Claims Policy is {}. \
    \nThe policy has a premium amount of {}. \
    \nThe Legal Auto Claims Policy must include the Address and Email Information at the beginning of the Legal Auto Claims Policy.  \
    \nThe Legal Auto Claims Policy must include the Policy Number at the beginning of the Legal Auto Claims Policy.  \
    \nThe Legal Auto Claims Policy must include the Start date and End date at the beginning of the Legal Auto Claims Policy.  \
    \nThe Legal Policy must have different subsections.\
    \nThe subsections are \
    a) Automobile Liability Insurance. b) Automobile Medical Payments. c) Automobile Debt Indemnity Insurance. \
    d) Uninsured Motorists Insurance. e) Default Provisions. f) Personal Injury Protection. g) Collision Insurance. h) Comprehensive Insurance. i) Rental Reimbursement Insurance. j) Towing and Labor Coverage.\
    k) Waiver of Deductible. Each subsection must include Exclusions. \
    \nEach subsection must include LIMITS OF LIABILITY and What is not covered. \
    \nTHe End the document must have closing remarks and place for signature.\
    \nDo NOT writing anything after the closing remarks and place for signature.\
    \nThe Legal Auto Claims Policy must also include the following mailing contact information {}, {},{} \
    ".format(company_name,primary_name,primary_address,primary_email_address,contract_id,begin_date, end_date,premium_amount, secondary_name,secondary_address,secondary_email_address)
    

In [19]:
print(prompt)

Compose a standard Legal Auto Claims Insurance Policy document between the Insurer State Farm Insurance and Michael Alexander \. The following are the details of the Insurer. 
Address: USNS Hansen
FPO AE 64250 
Email:   tiffanyoconnor@example.net. The Policy Number  for the Legal Auto Claims Policy is S2TCRs5GJQ0Urhd8. 
The Start date of the Legal Auto Claims Policy is 1909-1-22 and the End date of Legal Auto Claims Policy is 1918-12-30. 
The policy has a premium amount of 241023. 
The Legal Auto Claims Policy must include the Address and Email Information at the beginning of the Legal Auto Claims Policy.  
The Legal Auto Claims Policy must include the Policy Number at the beginning of the Legal Auto Claims Policy.  
The Legal Auto Claims Policy must include the Start date and End date at the beginning of the Legal Auto Claims Policy.  
The Legal Policy must have different subsections.
The subsections are a) Automobile Liability Insurance. b) Automobile Medical Payments. c) Automobile 

In [20]:
label_types= ['company_name','primary_name','primary_address','primary_email_address','contract_id','begin_date', 'end_date','premium_amount', 'secondary_name','secondary_address','secondary_email_address']

In [21]:
label_values = [company_name,primary_name,primary_address,primary_email_address,contract_id,begin_date, end_date,premium_amount, secondary_name,secondary_address,secondary_email_address]

In [22]:
from datetime import datetime
import re
import pandas as pd
print(datetime.now())
def model_seq_gen(model) : 
        pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer)
        start = datetime.now()
        sequences = pipe(
            f'{prompt}' ,
            do_sample=True,
            max_new_tokens=1200, return_full_text=False
            #temperature=0.7, 
            #top_p=0.95
        )
        extracted_title = sequences.copy()#re.sub(r'[\'"]', '', sequences[0]['generated_text'].split("[/INST]")[1])
        stop = datetime.now()
        time_taken = stop-start
        print(f"Execution Time : {time_taken}")
        return extracted_title
    
output_list = []
for i in range(1,2):
    extracted_title = model_seq_gen(model)
    text2 = extracted_title[0]['generated_text'].encode('latin-1', 'replace').decode('latin-1')

    output_list.append(text2)
    # save FPDF() class into a 
        # variable pdf
    pdf = FPDF()

        # Add a page
    pdf.add_page()

        # set style and size of font 
        # that you want in the pdf
    pdf.set_font("Arial", size = 15)

        # create a cell
    pdf.multi_cell(200, 10, txt = text2, 
                 align = 'C')


    # save the pdf with name .pdf
    pdf_filename = "output_"+ str(i) +"_v2a.pdf"
    #try:
    pdf.output(pdf_filename) 
    essay = label_values + [text2]
    #except:
    pass

pd.DataFrame(output_list).to_csv("synth_policies.csv")
print(extracted_title)
print(datetime.now())


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


2024-04-15 13:04:55.130605
Execution Time : 0:04:06.593265
[{'generated_text': "\nThe following paragraph is my Legal Auto Claims Policy for State Farm Insurance. \n\nPlease accept this as a formal, legal, and binding contract.\n\nAUTOMOBILE LIABILITY INSURANCE\nAutomobile Liability insurance is an agreement to provide payment for a legal obligation imposed by law for personal injury or death incurred to other people and for property damage to others. The maximum amount of automobile bodily injury benefits that the Insurer will provide for each person as a result of an accident is $100,000. The maximum amount of automobile bodily injury benefits that the Insurer will provide to any one accident is $300,000. The maximum amount of automobile property damage benefits that the Insurer will pay to any one person is $50,000. The maximum amount of automobile property damage benefits that the Insurer will pay in any one accident is $100,000. \n\nExclusions\nThis policy does not cover personal 

In [23]:
#information_list

In [24]:
print(extracted_title[0]['generated_text'])


The following paragraph is my Legal Auto Claims Policy for State Farm Insurance. 

Please accept this as a formal, legal, and binding contract.

AUTOMOBILE LIABILITY INSURANCE
Automobile Liability insurance is an agreement to provide payment for a legal obligation imposed by law for personal injury or death incurred to other people and for property damage to others. The maximum amount of automobile bodily injury benefits that the Insurer will provide for each person as a result of an accident is $100,000. The maximum amount of automobile bodily injury benefits that the Insurer will provide to any one accident is $300,000. The maximum amount of automobile property damage benefits that the Insurer will pay to any one person is $50,000. The maximum amount of automobile property damage benefits that the Insurer will pay in any one accident is $100,000. 

Exclusions
This policy does not cover personal injuries or damage to property caused by
1. The Insured's intentional acts. 
2. War or ac

In [25]:
print(text2)


The following paragraph is my Legal Auto Claims Policy for State Farm Insurance. 

Please accept this as a formal, legal, and binding contract.

AUTOMOBILE LIABILITY INSURANCE
Automobile Liability insurance is an agreement to provide payment for a legal obligation imposed by law for personal injury or death incurred to other people and for property damage to others. The maximum amount of automobile bodily injury benefits that the Insurer will provide for each person as a result of an accident is $100,000. The maximum amount of automobile bodily injury benefits that the Insurer will provide to any one accident is $300,000. The maximum amount of automobile property damage benefits that the Insurer will pay to any one person is $50,000. The maximum amount of automobile property damage benefits that the Insurer will pay in any one accident is $100,000. 

Exclusions
This policy does not cover personal injuries or damage to property caused by
1. The Insured's intentional acts. 
2. War or ac